# Calculating the speech transmission index from an impulse response

In [1]:
import numpy as np
import scipy.signal as signal
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.fft import fft
import math
from pyfilterbank import FractionalOctaveFilterbank
import utilities as util
import pandas as pd

## Step 1. Define the input
For the purpose of demonstration, let's set the input signal as the unit impulse function, padded out to 1.6 seconds long.

In [2]:
Fs = 48000  # Sample rate (Hz)
t = 1.6     # Period of sampling (seconds)
N = int(Fs*t)    # Total number of samples

impulse_response = signal.unit_impulse(N)

In [3]:
# Plotting the impulse response
x = np.arange(N)/Fs
y = impulse_response

fig_ir = go.FigureWidget()
fig_ir.add_scatter(x=x, y=y)
fig_ir.update_layout(title = {'text':'Impulse Response',
                              'xanchor':'center',
                              'x':0.5},
                     xaxis_title = 'Time (seconds)',
                     yaxis_title = 'Amplitude'
                     )
fig_ir

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '8f53a4ed-be07-4ed8-8b86-01e8b0d9a514',
              'x': array([0.00000000e+00, 2.08333333e-05, 4.16666667e-05, ..., 1.59993750e+00,
                          1.59995833e+00, 1.59997917e+00]),
              'y': array([1., 0., 0., ..., 0., 0., 0.])}],
    'layout': {'template': '...',
               'title': {'text': 'Impulse Response', 'x': 0.5, 'xanchor': 'center'},
               'xaxis': {'title': {'text': 'Time (seconds)'}},
               'yaxis': {'title': {'text': 'Amplitude'}}}
})

In [4]:
# Calculating the transfer function
transfer_function = fft(impulse_response)
transfer_function_magnitude = abs(transfer_function)
transfer_function_phase = np.angle(transfer_function, deg=True)

In [5]:
# Plotting the magnitude and phase response
x = np.arange(0,Fs/2,1/t)
y1 = transfer_function_magnitude[0:int(transfer_function.size/2)]
y2 = transfer_function_phase[0:int(transfer_function.size/2)]
fig_spectrum = go.FigureWidget()
fig_spectrum.add_scatter(x=x,y=y1,name='Magnitude')
fig_spectrum.add_scatter(x=x,y=y2,name='Phase')
fig_spectrum.update_layout(title = {'text':'Transfer Function',
                                    'xanchor':'center',
                                    'x':0.5},
                           xaxis_title = 'Frequency (Hertz)'
                           )
fig_spectrum.update_xaxes(type='log')
fig_spectrum

FigureWidget({
    'data': [{'name': 'Magnitude',
              'type': 'scatter',
              'uid': '0988df59-9640-4959-94f7-91479b16bca6',
              'x': array([0.0000000e+00, 6.2500000e-01, 1.2500000e+00, ..., 2.3998125e+04,
                          2.3998750e+04, 2.3999375e+04]),
              'y': array([1., 1., 1., ..., 1., 1., 1.])},
             {'name': 'Phase',
              'type': 'scatter',
              'uid': '21766a58-a26e-4683-944c-386a624ae9ae',
              'x': array([0.0000000e+00, 6.2500000e-01, 1.2500000e+00, ..., 2.3998125e+04,
                          2.3998750e+04, 2.3999375e+04]),
              'y': array([-0.,  0.,  0., ...,  0.,  0.,  0.])}],
    'layout': {'template': '...',
               'title': {'text': 'Transfer Function', 'x': 0.5, 'xanchor': 'center'},
               'xaxis': {'title': {'text': 'Frequency (Hertz)'}, 'type': 'log'}}
})

## Step 2: Filter Input into Octave Bands

In [6]:
filterbank = FractionalOctaveFilterbank(sample_rate=Fs, nth_oct=1, order=4, start_band = -3, end_band = 3, edge_correction_percent=-4.25) # Note that band 0 is 1kHz, and band number is spaced 1/nth_oct apart.

ir_octavebandfiltered, states = filterbank.filter(impulse_response, ffilt=True)

fig_ir = go.FigureWidget()

x = np.arange(0,ir_octavebandfiltered.shape[0]/Fs,1/Fs)
for index in np.arange(ir_octavebandfiltered.shape[1]):
    y = ir_octavebandfiltered[:,index]
    fig_ir.add_scatter(x=x, y=y, name=int(filterbank.center_frequencies[index]))

fig_ir.update_layout(title = {'text':'Octave-band Filtered Impulse Response',
                              'xanchor':'center',
                              'x':0.5},
                     xaxis_title = 'Time (seconds)',
                     yaxis_title = 'Amplitude'
                     )
fig_ir.update_xaxes(range=[0,0.1])
fig_ir

FigureWidget({
    'data': [{'name': '125',
              'type': 'scatter',
              'uid': '6424af2a-f6fe-4e50-9387-b359363dce86',
              'x': array([0.00000000e+00, 2.08333333e-05, 4.16666667e-05, ..., 1.59993750e+00,
                          1.59995833e+00, 1.59997917e+00]),
              'y': array([5.10693677e-09, 2.03394110e-08, 5.56445644e-08, ..., 5.68925268e-50,
                          5.60691639e-50, 5.52396100e-50])},
             {'name': '250',
              'type': 'scatter',
              'uid': '76490c70-65db-4e2f-a26e-a9073009ba76',
              'x': array([0.00000000e+00, 2.08333333e-05, 4.16666667e-05, ..., 1.59993750e+00,
                          1.59995833e+00, 1.59997917e+00]),
              'y': array([ 8.03494864e-08,  3.18488796e-07,  8.66288631e-07, ..., -1.69179477e-96,
                          -1.60614487e-96, -1.52000131e-96])},
             {'name': '500',
              'type': 'scatter',
              'uid': '68cae242-fdbc-4579-8d89-7e1

In [7]:
x = np.arange(0,Fs/2,1/t)

fig_tf_octavebandfiltered = go.FigureWidget()

for index in np.arange(ir_octavebandfiltered.shape[1]):
    y = 20*np.log10(abs(fft(ir_octavebandfiltered[:,index])))[0:len(x)]
    fig_tf_octavebandfiltered.add_scatter(x=x, y=y, name=int(filterbank.center_frequencies[index]))

y = 10*np.log10(abs(fft(np.sum(ir_octavebandfiltered,axis=1)))[0:len(x)])
fig_tf_octavebandfiltered.add_scatter(x=x, y=y, name='Cumulative', line=dict(width=6,dash='dot'))

fig_tf_octavebandfiltered.update_layout(title = {'text':'Transfer Function (Octave Bands)',
                                    'xanchor':'center',
                                    'x':0.5},
                           xaxis_title = 'Frequency (Hertz)'
                           )
fig_tf_octavebandfiltered.update_xaxes(type='log')
fig_tf_octavebandfiltered.update_xaxes(range=[np.log10(20),np.log10(20000)])
fig_tf_octavebandfiltered.update_yaxes(range=[-60,0])
fig_tf_octavebandfiltered

FigureWidget({
    'data': [{'name': '125',
              'type': 'scatter',
              'uid': '5a12be29-a761-4d2c-84f9-1c77f46c2212',
              'x': array([0.0000000e+00, 6.2500000e-01, 1.2500000e+00, ..., 2.3998125e+04,
                          2.3998750e+04, 2.3999375e+04]),
              'y': array([-183.86871308, -185.20306302, -181.72807018, ..., -183.86734219,
                          -183.86733369, -183.86732848])},
             {'name': '250',
              'type': 'scatter',
              'uid': '542cdd56-5257-4f8d-bd9a-5a6630802dff',
              'x': array([0.0000000e+00, 6.2500000e-01, 1.2500000e+00, ..., 2.3998125e+04,
                          2.3998750e+04, 2.3999375e+04]),
              'y': array([-159.90002376, -159.90492157, -159.97872614, ..., -159.90002781,
                          -159.90002775, -159.9000277 ])},
             {'name': '500',
              'type': 'scatter',
              'uid': '6d7b6995-fe3b-40b8-a1dc-d2c63559f185',
              'x':

## Step 3: Calculate the Modulation Transfer Function


Calculate the energy of the signal in each octave band:

In [8]:
energy = {}

for index in np.arange(ir_octavebandfiltered.shape[1]):
    energy.update( {int(filterbank.center_frequencies[index]):(ir_octavebandfiltered[:,index]**2).sum()} )

Calculate the modulation transfer function at each octave band:

In [9]:
mtf = {}

for index in np.arange(ir_octavebandfiltered.shape[1]):
    mtf_complex = fft((ir_octavebandfiltered[:,index]**2)/energy[int(filterbank.center_frequencies[index])])
    mtf_magnitude = abs(mtf_complex)
    mtf_phase = np.angle(mtf_complex)
    mtf.update( {int(filterbank.center_frequencies[index]): {'complex':mtf_complex,
                                                             'magnitude':mtf_magnitude,
                                                             'phase':mtf_phase
    }} )

In [10]:
fig_mtf = go.FigureWidget()

x = np.arange(0,Fs,1/t)

# Find maximum x value to be parsed to the plotting routine to reduce calculation time.
closest_match = util.find_nearest(x,12.5)
x_max = np.where(x==closest_match)[0][0] # [0][0] is to convert array to value.

# Generate plot
for index in np.arange(ir_octavebandfiltered.shape[1]):
    y = mtf[int(filterbank.center_frequencies[index])]['magnitude']

    fig_mtf.add_trace(go.Scatter(x=x[0:x_max],y=y[0:x_max],mode='markers',name='Mag @ '+str(int(filterbank.center_frequencies[index]))))
    fig_mtf.update_layout(    title = {'text':'Modulation Transfer Function',
                                       'xanchor':'center',
                                       'x':0.5},
                              xaxis_title = 'Frequency (Hertz)')
fig_mtf.update_xaxes(range=[np.log10(0.625),np.log10(12.5)],type='log')
fig_mtf

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'Mag @ 125',
              'type': 'scatter',
              'uid': 'fc1c8862-503f-4737-bf17-0441b2ee6ff9',
              'x': array([ 0.   ,  0.625,  1.25 ,  1.875,  2.5  ,  3.125,  3.75 ,  4.375,  5.   ,
                           5.625,  6.25 ,  6.875,  7.5  ,  8.125,  8.75 ,  9.375, 10.   , 10.625,
                          11.25 , 11.875]),
              'y': array([1.        , 0.99978314, 0.99913388, 0.99805611, 0.99655623, 0.994643  ,
                          0.99232737, 0.98962226, 0.98654227, 0.98310345, 0.979323  , 0.97521899,
                          0.97081006, 0.96611517, 0.96115336, 0.95594353, 0.95050423, 0.9448535 ,
                          0.93900876, 0.93298667])},
             {'mode': 'markers',
              'name': 'Mag @ 250',
              'type': 'scatter',
              'uid': '42b3d8e3-e9c7-41f5-ad46-1afac9964772',
              'x': array([ 0.   ,  0.625,  1.25 ,  1.875,  2.5  ,  3.125, 

In [11]:
closest_match = util.find_nearest(x,12.5)
x_max = np.where(x==closest_match)
x_max[0][0]

20

In [12]:
mtf_table_dict = {125:{},250:{},500:{},1000:{},2000:{},4000:{},8000:{}} # Initialise mtf table dictionary
for oct_band in mtf:
    for modulation_freq in [0.63, 0.8, 1.0, 1.25, 1.6, 2.0, 2.5, 3.15, 4.0, 5.0, 6.3, 8.0, 10.0, 12.5]:
        closest_match = util.find_nearest(x,modulation_freq)
        position = np.where(x==closest_match)
        mtf_table_dict[oct_band][modulation_freq]=mtf[oct_band]['magnitude'][position][0]


In [13]:
pd.set_option('display.float_format', '{:.4f}'.format)
mtf_df = pd.DataFrame(mtf_table_dict)
mtf_df

,125,250,500,1000,2000,4000,8000
0.6300,0.9998,0.9999,1.0000,1.0000,1.0000,1.0000,1.0000
0.8000,0.9998,0.9999,1.0000,1.0000,1.0000,1.0000,1.0000
1.0000,0.9991,0.9998,0.9999,1.0000,1.0000,1.0000,1.0000
1.2500,0.9991,0.9998,0.9999,1.0000,1.0000,1.0000,1.0000
1.6000,0.9981,0.9995,0.9999,1.0000,1.0000,1.0000,1.0000
2.0000,0.9981,0.9995,0.9999,1.0000,1.0000,1.0000,1.0000
2.5000,0.9966,0.9991,0.9998,0.9999,1.0000,1.0000,1.0000
3.1500,0.9946,0.9986,0.9997,0.9999,1.0000,1.0000,1.0000
4.0000,0.9923,0.9981,0.9995,0.9999,1.0000,1.0000,1.0000
5.0000,0.9865,0.9966,0.9991,0.9998,0.9999,1.0000,1.0000


# Work-In-Progress Below

In [14]:
# Calculating the Level dependent auditory masking

level_of_adjacent_lower_octave_band = 9 #L_k-1 - PLACEHOLDER
intensity_of_adjacent_lower_octave_band = 10**(level_of_adjacent_lower_octave_band/10) # I_k-1

if level_of_adjacent_lower_octave_band < 63:
    auditory_masking_dB = 0.5 * level_of_adjacent_lower_octave_band - 65
elif level_of_adjacent_lower_octave_band >= 63 and level_of_adjacent_lower_octave_band < 67:
    auditory_masking_dB = 1.8 * level_of_adjacent_lower_octave_band - 146.9
elif level_of_adjacent_lower_octave_band >= 67 and level_of_adjacent_lower_octave_band < 100:
    auditory_masking_dB = 0.5 * level_of_adjacent_lower_octave_band - 59.8
elif level_of_adjacent_lower_octave_band >= 100:
    auditory_masking_dB = -10

auditory_masking_factor = 10**(auditory_masking_dB/10) #amf

masking_intensity = intensity_of_adjacent_lower_octave_band * auditory_masking_factor #I_am,k


In [15]:
# Absolute speech reception threshold
ABSOLUTE_SPEECH_RECEPTION_THRESHOLD = {125:46,250:27,500:12,1000:6.5,2000:7.5,4000:8,8000:12} # also known as ART
RECEPTION_THRESHOLD_INTENSITY = {125:10**(46/10),250:10**(27/10),500:10**(12/10),1000:10**(6.5/10),2000:10**(7.5/10),4000:8,8000:10**(12/10)}

In [16]:
# Gender-specific octave band weighting and redundancy factors - Table A.3 of 60268-16 IEC2011(E)
MTI_OCTAVE_BAND_WEIGHTING_FACTORS = {
    'male':{
        'alpha':{125:0.085,250:0.127,500:0.230,1000:0.233,2000:0.309,4000:0.224,8000:0.173},
        'beta': {125:0.085,250:0.078,500:0.065,1000:0.011,2000:0.047,4000:0.095,8000:None}
    },
    'female':{
        'alpha':{125:None,250:0.117,500:0.223,1000:0.216,2000:0.328,4000:0.250,8000:0.194},
        'beta': {125:None,250:0.099,500:0.066,1000:0.062,2000:0.025,4000:0.076,8000:None}
    }
}


In [17]:
# Gender-specific spectra of STI test signals
# Note - These are related to speech spectra?
STI_TEST_SIGNAL_OCTAVE_BAND_LEVELS = {
    'male':{125:2.9, 250:2.9, 500:-0.8, 1000:-6.8, 2000:-12.8, 4000:-18.8, 8000:-24.8},
    'female':{125:None, 250:5.3, 500:-1.9, 1000:-9.1, 2000:-15.8, 4000:-16.7, 8000:-18.0}
}
